In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import traceback

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Import the CSV created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,0,Winslow,US,2020-04-19 19:49:56,35.02,-110.70,66.20,15,1,12.91,clear sky,0.0,0.00
1,1,Hilo,US,2020-04-19 19:56:18,19.73,-155.09,78.80,57,75,3.36,broken clouds,0.0,0.00
2,2,Provideniya,RU,2020-04-19 19:54:23,64.38,-173.30,20.86,95,100,15.19,light snow,0.0,0.56
3,3,Chuy,UY,2020-04-19 19:57:14,-33.70,-53.46,67.73,78,4,8.21,clear sky,0.0,0.00
4,4,Uthal,PK,2020-04-19 19:59:32,25.81,66.62,77.49,77,93,2.80,overcast clouds,0.0,0.00


In [4]:
# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
# Prompt the customer for the maximum temperature preference.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Prompt the customer to answer if he or she would like it to be raining or not.
current_rain = input("Do you want it to be raining? (yes/no) ")
# Prompt the customer to answer if he or she would like it to be snowing or not.
current_snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 82
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
if current_rain == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3hrs)"] > 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3hrs)"] == 0)]
    
if current_snow == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Snow inches (last 3hrs)"] > 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Snow inches (last 3hrs)"] == 0)]

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
1,1,Hilo,US,2020-04-19 19:56:18,19.73,-155.09,78.80,57,75,3.36,broken clouds,0.0,0.0
4,4,Uthal,PK,2020-04-19 19:59:32,25.81,66.62,77.49,77,93,2.80,overcast clouds,0.0,0.0
7,7,Bambous Virieux,MU,2020-04-19 19:59:33,-20.34,57.76,77.00,78,40,10.29,scattered clouds,0.0,0.0
8,8,Caravelas,BR,2020-04-19 19:59:33,-17.71,-39.25,76.10,76,46,17.65,scattered clouds,0.0,0.0
14,14,Vaini,TO,2020-04-19 19:59:35,-21.20,-175.20,78.80,83,20,8.05,few clouds,0.0,0.0
17,17,Arraial Do Cabo,BR,2020-04-19 19:59:36,-22.97,-42.03,77.00,65,40,8.05,scattered clouds,0.0,0.0
32,32,Warri,NG,2020-04-19 19:59:40,5.52,5.75,78.01,89,100,1.19,overcast clouds,0.0,0.0
34,34,Port Alfred,ZA,2020-04-19 19:59:40,-33.59,26.89,79.66,45,26,12.66,scattered clouds,0.0,0.0
35,35,Kupang,ID,2020-04-19 19:59:41,-10.17,123.58,80.60,74,41,10.40,scattered clouds,0.0,0.0
36,36,Vila Velha,BR,2020-04-19 19:58:31,-20.33,-40.29,78.01,69,40,14.99,scattered clouds,0.0,0.0


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,78.80,broken clouds,19.73,-155.09,
4,Uthal,PK,77.49,overcast clouds,25.81,66.62,
7,Bambous Virieux,MU,77.00,scattered clouds,-20.34,57.76,
8,Caravelas,BR,76.10,scattered clouds,-17.71,-39.25,
14,Vaini,TO,78.80,few clouds,-21.20,-175.20,
17,Arraial Do Cabo,BR,77.00,scattered clouds,-22.97,-42.03,
32,Warri,NG,78.01,overcast clouds,5.52,5.75,
34,Port Alfred,ZA,79.66,scattered clouds,-33.59,26.89,
35,Kupang,ID,80.60,scattered clouds,-10.17,123.58,
36,Vila Velha,BR,78.01,scattered clouds,-20.33,-40.29,


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        traceback.print_exc()
        print("Hotel not found... skipping.")

Traceback (most recent call last):
  File "<ipython-input-7-17fca176cdd4>", line 22, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range


Hotel not found... skipping.


Traceback (most recent call last):
  File "<ipython-input-7-17fca176cdd4>", line 22, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range


Hotel not found... skipping.


Traceback (most recent call last):
  File "<ipython-input-7-17fca176cdd4>", line 22, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range


...

Hotel not found... skipping.


Traceback (most recent call last):
  File "<ipython-input-7-17fca176cdd4>", line 22, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range


Hotel not found... skipping.


Traceback (most recent call last):
  File "<ipython-input-7-17fca176cdd4>", line 22, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range


Hotel not found... skipping.


In [8]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,78.80,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
4,Uthal,PK,77.49,overcast clouds,25.81,66.62,Al Madina Hotal
7,Bambous Virieux,MU,77.00,scattered clouds,-20.34,57.76,Casa Tia Villa
8,Caravelas,BR,76.10,scattered clouds,-17.71,-39.25,Pousada dos Navegantes
14,Vaini,TO,78.80,few clouds,-21.20,-175.20,Keleti Beach Resort


In [9]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Create a marker layer map for the vacation spots with a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))